In [20]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import PIL
from tensorflow.keras import datasets, layers, models
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D,Activation, Flatten, Dropout, Dense
from keras_tuner import HyperModel, HyperParameters, RandomSearch
import keras_tuner as kt
from keras.callbacks import ModelCheckpoint

In [2]:
from PIL import Image
import os

# create data generators

In [3]:
target_size = (96,96)

In [25]:
batch_size = 128

In [26]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range = .3,
    zoom_range = .3,
    horizontal_flip=True,
    brightness_range = [.4,1.5],
    rotation_range = 30,
    width_shift_range=0.15,
    height_shift_range=0.15
    #featurewise_std_normalization=False # not sure how this will work here
)

In [27]:
train_generator = train_datagen.flow_from_directory(
        r'C:/Users/natha/OneDrive/Desktop/datascience/machine learning and predictive/final/dataset/output/train',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 32938 images belonging to 151 classes.


In [28]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [29]:
test_data_generator = test_datagen.flow_from_directory(
    r'C:/Users/natha/OneDrive/Desktop/datascience/machine learning and predictive/final/dataset/output/val',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 8314 images belonging to 151 classes.


# tune hyper parameters

In [9]:
#,validation_data = test_data_generator

In [10]:
class MyHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):

        # Define the hyperparameter search space for Conv2D
        padding = hp.Choice('padding', values = ['same', 'valid'])
        kernel_initializer = hp.Choice('kernel_initializer', values = ['glorot_normal', 'he_normal'])
        # Add the Conv2D layer with tunable hyperparameters
        
        model = keras.Sequential()
        model.add(Conv2D(32, 3, padding = padding, activation = 'relu', input_shape =(96, 96, 3), kernel_initializer = kernel_initializer))
        model.add(BatchNormalization(axis = -1))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, 3, padding = padding, kernel_initializer = kernel_initializer, activation = 'relu'))
        model.add(BatchNormalization(axis = -1))
        model.add(Conv2D(64, 3, padding = padding, kernel_initializer = kernel_initializer, activation = 'relu'))
        model.add(BatchNormalization(axis = -1))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(128, 3, padding = padding, kernel_initializer = kernel_initializer, activation = 'relu'))
        model.add(BatchNormalization(axis = -1))
        model.add(Conv2D(128, 3, padding = padding, kernel_initializer = kernel_initializer, activation = 'relu'))
        model.add(BatchNormalization(axis = -1))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(256, 3, padding = padding, kernel_initializer = kernel_initializer, activation = 'relu'))
        model.add(BatchNormalization(axis = -1))
        model.add(Conv2D(256, 3, padding = padding, kernel_initializer = kernel_initializer, activation = 'relu'))
        model.add(BatchNormalization(axis = -1))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512, activation = 'relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(256, activation = 'relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(151, activation = 'softmax'))

        
        model.compile(
            optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,.001])),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model



In [11]:
# Specify the input shape and number of classes
input_shape = (96, 96, 3)
num_classes = 151

# Create an instance of the custom HyperModel
hypermodel = MyHyperModel(input_shape, num_classes)

# Create a HyperParameters object
#hyperparameters = HyperParameters()

# Define the search space for filters and kernel_size
#hyperparameters.Int('filters', min_value=64, max_value=128, step=32)
#hyperparameters.Choice('kernel_size', values=[3, 5])
#hyperparameters.Choice('pool_size', values=[2, 3])


In [12]:
# Create a RandomSearch tuner with the hypermodel and hyperparameters
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=8,
    #hyperparameters=hyperparameters,
    directory='my_directory_advanced',  # Specify a directory to store the tuner results
    project_name='my_project'  # Specify a project name for the tuner
)



In [14]:
# Perform the hyperparameter search
tuner.search(train_generator, epochs=4,steps_per_epoch = 16, validation_data=(test_data_generator))



Trial 8 Complete [00h 17m 18s]
val_accuracy: 0.007216742727905512

Best val_accuracy So Far: 0.010343997739255428
Total elapsed time: 02h 23m 07s
INFO:tensorflow:Oracle triggered exit


In [15]:
# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

In [16]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

In [17]:
print(best_hyperparameters.get('padding'))
print(best_hyperparameters.get('kernel_initializer'))
print(best_hyperparameters.get('learning_rate'))

same
he_normal
0.01


In [18]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 112, 112, 64)     2